In [ ]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Загрузка датасета с разбиением данных 1000/500
train_features_partly = pd.read_csv('dataset/partly/x_train.csv')
train_labels_partly = pd.read_csv('dataset/partly/y_train.csv')
test_features_partly = pd.read_csv('dataset/partly/x_check.csv')
test_labels_partly = pd.read_csv('dataset/partly/y_check.csv')

In [ ]:
# Загрузка датасета с рандомным разбиением данных
train_features_random = pd.read_csv('dataset/random/x_train.csv')
train_labels_random = pd.read_csv('dataset/random/y_train.csv')
test_features_random = pd.read_csv('dataset/random/x_check.csv')
test_labels_random = pd.read_csv('dataset/random/y_check.csv')

In [ ]:
# Загрузка датасета, где 19 моделей идут на обучение, а оставшиеся на проверку
train_features_someforlearning = pd.read_csv('dataset/someforlearning/x_train.csv')
train_labels_someforlearning = pd.read_csv('dataset/someforlearning/y_train.csv')
test_features_someforlearning = pd.read_csv('dataset/someforlearning/x_check.csv')
test_labels_someforlearning = pd.read_csv('dataset/someforlearning/y_check.csv')

In [ ]:
# Визуализация данных
col_target = ['area', 'delay','area*delay']

fig1 = plt.figure(figsize = (20, 30))
fig2 = plt.figure(figsize = (20, 30))
fig3 = plt.figure(figsize = (20, 30))
for i , v in enumerate(col_target):
  axes = fig1.add_subplot(9, 3, i+1)
  sns.histplot(train_labels_partly[v], ax = axes, kde=True, stat="density")

for i , v in enumerate(col_target):
  axes = fig2.add_subplot(9, 3, i+1)
  sns.histplot(train_labels_random[v], ax = axes, kde=True, stat="density")

for i , v in enumerate(col_target):
  axes = fig3.add_subplot(9, 3, i+1)
  sns.histplot(train_labels_someforlearning[v], ax = axes, kde=True, stat="density")

In [ ]:
col = ['BUFF', 'NOT', 'AND', 'PI', 'PO', 'LP']

fig1 = plt.figure(figsize = (20, 30))
fig2 = plt.figure(figsize = (20, 30))
fig3 = plt.figure(figsize = (20, 30))
for i , v in enumerate(col):
  axes = fig1.add_subplot(9, 3, i+1)
  sns.histplot(train_features_partly[v], ax = axes, kde=True, stat="density")

for i , v in enumerate(col):
  axes = fig2.add_subplot(9, 3, i+1)
  sns.histplot(train_features_random[v], ax = axes, kde=True, stat="density")

for i , v in enumerate(col):
  axes = fig3.add_subplot(9, 3, i+1)
  sns.histplot(train_features_someforlearning[v], ax = axes, kde=True, stat="density")

In [ ]:
# Оптимизация гиперпараметров при предсказании area через RandomizedSearch на датасете partly

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,0.5,0.002),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1)
}

mape=1
i=0
while mape >0.011 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_partly, train_labels_partly['area'])

    # Получение метрик
    predictions = random_search.predict(test_features_partly)
    mape = mean_absolute_percentage_error(test_labels_partly['area'], predictions)
    r2 = r2_score(test_labels_partly['area'], predictions)
    mse = mean_squared_error(test_labels_partly['area'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании area через RandomizedSearch на датасете random

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
    
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,0.5,0.002),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1)
}
mape=1
i=0
while mape >0.0117 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_random, train_labels_random['area'])

    # Получение метрик
    predictions = random_search.predict(test_features_random)
    mape = mean_absolute_percentage_error(test_labels_random['area'], predictions)
    r2 = r2_score(test_labels_random['area'], predictions)
    mse = mean_squared_error(test_labels_random['area'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании area через RandomizedSearch на датасете someforlearning

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,0.5,0.002),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1)
}
mape=1
i=1138
while mape >0.1789 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_someforlearning, train_labels_someforlearning['area'])

    # Получение метрик
    predictions = random_search.predict(test_features_someforlearning)
    mape = mean_absolute_percentage_error(test_labels_someforlearning['area'], predictions)
    r2 = r2_score(test_labels_someforlearning['area'], predictions)
    mse = mean_squared_error(test_labels_someforlearning['area'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании delay через RandomizedSearch на датасете partly

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,0.5,0.002),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1)
}
mape=1
i=8
while mape >0.152 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_partly, train_labels_partly['delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_partly)
    mape = mean_absolute_percentage_error(test_labels_partly['delay'], predictions)
    r2 = r2_score(test_labels_partly['delay'], predictions)
    mse = mean_squared_error(test_labels_partly['delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании delay через RandomizedSearch на датасете random

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,0.5,0.002),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1)
}
i=24
mape=1
while mape >0.156 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )
    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_random, train_labels_random['delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_random)
    mape = mean_absolute_percentage_error(test_labels_random['delay'], predictions)
    r2 = r2_score(test_labels_random['delay'], predictions)
    mse = mean_squared_error(test_labels_random['delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании delay через RandomizedSearch на датасете someforlearning

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.001,1,0.005),
    'n_estimators': np.arange(40, 150, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.01,1.0,0.01),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1),
    
}
mape=1
i=0
r2=-1
while r2<0.4 or mape >0.65:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=1,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_someforlearning, train_labels_someforlearning['delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_someforlearning)
    mape = mean_absolute_percentage_error(test_labels_someforlearning['delay'], predictions)
    r2 = r2_score(test_labels_someforlearning['delay'], predictions)
    mse = mean_squared_error(test_labels_someforlearning['delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании area*delay через RandomizedSearch на датасете partly
# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.01,0.5,0.02),
    'n_estimators': np.arange(40, 100, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.1,1.0,0.1),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1),
    'rate_drop': np.arange(0,1,0.01)
}
mape=1
i=0
while mape >0.150 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=0,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_partly, train_labels_partly['area*delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_partly)
    mape = mean_absolute_percentage_error(test_labels_partly['area*delay'], predictions)
    r2 = r2_score(test_labels_partly['area*delay'], predictions)
    mse = mean_squared_error(test_labels_partly['area*delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании area*delay через RandomizedSearch на датасете random

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.01,0.5,0.02),
    'n_estimators': np.arange(40, 100, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.1,1.0,0.1),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1),
    'rate_drop': np.arange(0,1,0.01)
}
mape=1
i=0
while mape >0.16 and i != 2000:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=0,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_random, train_labels_random['area*delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_random)
    mape = mean_absolute_percentage_error(test_labels_random['area*delay'], predictions)
    r2 = r2_score(test_labels_random['area*delay'], predictions)
    mse = mean_squared_error(test_labels_random['area*delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Оптимизация гиперпараметров при предсказании area*delay через RandomizedSearch на датасете someforlearning

# Инициализация модели
model = xgb.XGBRegressor(
    booster="dart"
)

# Определение сетки параметров
param_grid = {
    'learning_rate': np.arange(0.01,0.5,0.02),
    'n_estimators': np.arange(40, 100, 1),
    'max_depth': np.arange(3,11,1),
    'subsample': np.arange(0.1,1.0,0.1),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_lambda': np.arange(0,1,0.1),
    'rate_drop': np.arange(0,1,0.01)
}
mape=1
i=13816
r2=-1
while r2<0.5 or mape >0.50:
    # Создание экземпляра RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,  # Количество итераций случайного поиска
        scoring='neg_mean_absolute_percentage_error',
        cv=2,
        verbose=0,
        random_state=i
    )

    # Обучение модели с случайным поиском параметров
    random_search.fit(train_features_someforlearning, train_labels_someforlearning['area*delay'])

    # Получение метрик
    predictions = random_search.predict(test_features_someforlearning)
    mape = mean_absolute_percentage_error(test_labels_someforlearning['area*delay'], predictions)
    r2 = r2_score(test_labels_someforlearning['area*delay'], predictions)
    mse = mean_squared_error(test_labels_someforlearning['area*delay'], predictions)
    print(i)
    print('Best parameters: '+ str(random_search.best_params_))
    print('MAPE: '+ str(mape))
    print('R2: '+ str(r2))
    print('MSE: '+ str(mse)+'\n')
    i+=1
print('Best parameters: '+ str(random_search.best_params_))
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади для 1000/500

best_params_area= {'subsample': 0.23, 'reg_lambda': 0.7000000000000001, 'n_estimators': 57, 'max_depth': 10, 'learning_rate': 0.111, 'colsample_bytree': 0.9}
print(str(best_params_area)+'\n')

# Инициализация модели с Dart
model_area_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_area  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_area_dart.fit(train_features_partly, train_labels_partly['area'])
predictions = model_area_dart.predict(test_features_partly)
mape = mean_absolute_percentage_error(test_labels_partly['area'], predictions)
r2=r2_score(test_labels_partly['area'], predictions)
mse = mean_squared_error(test_labels_partly['area'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади рандом

best_params_area= {'subsample': 0.23, 'reg_lambda': 0.7000000000000001, 'n_estimators': 57, 'max_depth': 10, 'learning_rate': 0.111, 'colsample_bytree': 0.9}

print(str(best_params_area)+'\n')

# Инициализация модели с Dart
model_area_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_area  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_area_dart.fit(train_features_random, train_labels_random['area'])
predictions = model_area_dart.predict(test_features_random)
mape = mean_absolute_percentage_error(test_labels_random['area'], predictions)
r2=r2_score(test_labels_random['area'], predictions)
mse = mean_squared_error(test_labels_random['area'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади 19/10

best_params_area= {'subsample': 0.14, 'reg_lambda': 0.1, 'n_estimators': 66, 'max_depth': 6, 'learning_rate': 0.02099999999999999, 'colsample_bytree': 0.7100000000000001}
print(str(best_params_area)+'\n')

# Инициализация модели с Dart
model_area_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_area  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_area_dart.fit(train_features_someforlearning, train_labels_someforlearning['area'])
predictions = model_area_dart.predict(test_features_someforlearning)
mape = mean_absolute_percentage_error(test_labels_someforlearning['area'], predictions)
r2=r2_score(test_labels_someforlearning['area'], predictions)
mse = mean_squared_error(test_labels_someforlearning['area'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание задержки для 1000/500

best_params_delay = {'subsample': 0.75, 'reg_lambda': 0.4, 'n_estimators': 95, 'max_depth': 7, 'learning_rate': 0.22999999999999982, 'colsample_bytree': 0.7100000000000001}

print(str(best_params_delay)+'\n')

# Инициализация модели с Dart
model_delat_dart = xgb.XGBRegressor(
    booster="dart",
    rate_drop = 0.19,
    **best_params_delay  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_delat_dart.fit(train_features_partly, train_labels_partly['delay'])
predictions = model_delat_dart.predict(test_features_partly)
mape = mean_absolute_percentage_error(test_labels_partly['delay'], predictions)
r2=r2_score(test_labels_partly['delay'], predictions)
mse = mean_squared_error(test_labels_partly['delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание задержки рандом

best_params_delay = {'subsample': 0.56, 'reg_lambda': 0.7000000000000001, 'n_estimators': 63, 'max_depth': 7, 'learning_rate': 0.1429999999999999, 'colsample_bytree': 0.81}


print(str(best_params_delay)+'\n')

# Инициализация модели с Dart
model_delat_dart = xgb.XGBRegressor(
    booster="dart",
    rate_drop = 0.13,
    **best_params_delay  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_delat_dart.fit(train_features_random, train_labels_random['delay'])
predictions = model_delat_dart.predict(test_features_random)
mape = mean_absolute_percentage_error(test_labels_random['delay'], predictions)
r2=r2_score(test_labels_random['delay'], predictions)
mse = mean_squared_error(test_labels_random['delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание задержки 19/10

best_params_delay =  {'subsample': 0.6900000000000001, 'reg_lambda': 0.4, 'n_estimators': 62, 'max_depth': 4, 'learning_rate': 0.706, 'colsample_bytree': 0.2}


print(str(best_params_delay)+'\n')

# Инициализация модели с Dart
model_delat_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_delay  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели
model_delat_dart.fit(train_features_someforlearning, train_labels_someforlearning['delay'])
predictions = model_delat_dart.predict(test_features_someforlearning)
mape = mean_absolute_percentage_error(test_labels_someforlearning['delay'], predictions)
r2=r2_score(test_labels_someforlearning['delay'], predictions)
mse = mean_squared_error(test_labels_someforlearning['delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади на задержку для 1000/500

best_params_all=  {'subsample': 0.9, 'reg_lambda': 0.4, 'rate_drop': 0.22, 'n_estimators': 60, 'max_depth': 10, 'learning_rate': 0.18999999999999997, 'colsample_bytree': 0.8}


print(str(best_params_all)+'\n')

# Инициализация модели с Dart
model_all_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_all  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели, предсказание, получение данных по метрикам
model_all_dart.fit(train_features_partly, train_labels_partly['area*delay'])
predictions = model_all_dart.predict(test_features_partly)
mape = mean_absolute_percentage_error(test_labels_partly['area*delay'], predictions)
r2=r2_score(test_labels_partly['area*delay'], predictions)
mse = mean_squared_error(test_labels_partly['area*delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади на задержку рандом

best_params_all= {'subsample': 0.9, 'reg_lambda': 0.4, 'rate_drop': 0.22, 'n_estimators': 60, 'max_depth': 10, 'learning_rate': 0.18999999999999997, 'colsample_bytree': 0.8}

print(str(best_params_all)+'\n')

# Инициализация модели с Dart
model_all_dart = xgb.XGBRegressor(
    booster="dart",
    **best_params_all  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели, предсказание, получение данных по метрикам
model_all_dart.fit(train_features_random, train_labels_random['area*delay'])
predictions = model_all_dart.predict(test_features_random)
mape = mean_absolute_percentage_error(test_labels_random['area*delay'], predictions)
r2=r2_score(test_labels_random['area*delay'], predictions)
mse = mean_squared_error(test_labels_random['area*delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')

In [ ]:
# Предсказание площади на задержку 19/10


best_params_all= {'subsample': 0.45000000000000007, 'n_estimators': 1, 'max_depth': 8, 'learning_rate': 0.37, 'colsample_bytree': 0.7}

print(str(best_params_all)+'\n')

# Инициализация модели с Dart
model_all_dart = xgb.XGBRegressor(
    booster="dart",
    reg_lambda=0,
    **best_params_all  # передайте лучшие параметры в качестве именованных аргументов
)
# Обучение модели, предсказание, получение данных по метрикам
model_all_dart.fit(train_features_someforlearning, train_labels_someforlearning['area*delay'])
predictions = model_all_dart.predict(test_features_someforlearning)
mape = mean_absolute_percentage_error(test_labels_someforlearning['area*delay'], predictions)
r2=r2_score(test_labels_someforlearning['area*delay'], predictions)
mse = mean_squared_error(test_labels_someforlearning['area*delay'], predictions)

print('Model with Dart booster:')
print('MAPE: '+ str(mape))
print('R2: '+ str(r2))
print('MSE: '+ str(mse)+'\n')